In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from torch.utils.data import Dataset, DataLoader, random_split
from prepare_sn_dataset import SNDataset


# CNN模型定义
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(32 * (window_length // 4), 128)
        self.relu3 = nn.ReLU()
        self.fc2 = nn.Linear(128, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = x.view(x.size(0), -1)
        x = self.fc1(x)
        x = self.relu3(x)
        x = self.fc2(x)
        return x

# 训练函数
def train_model(model, train_dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    
    for waveform, label in train_dataloader:
        waveform = waveform.to(device)
        label = label.to(device)
        
        optimizer.zero_grad()
        
        outputs = model(waveform.unsqueeze(1))
        loss = criterion(outputs, label)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    return running_loss / len(train_dataloader)

# 推断函数
def inference(model, test_dataloader, device):
    model.eval()
    predictions = []
    
    with torch.no_grad():
        for waveform, _ in test_dataloader:
            waveform = waveform.to(device)
            
            outputs = model(waveform.unsqueeze(1))
            _, predicted = torch.max(outputs.data, 1)
            
            predictions.extend(predicted.tolist())
    
    return predictions

# 设置训练参数
batch_size = 32
num_epochs = 10
learning_rate = 0.001
window_length = 5  # 窗口长度为5秒

# 定义训练和测试数据集
root_pth_speech = "/home/xinying/xinying/separated/speech"
root_pth_noise = "/home/xinying/xinying/separated/noise"
dataset = SNDataset(root_pth_speech,
                 root_pth_noise,)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size 
train_set, test_set = random_split(
    dataset=dataset, lengths=[train_size,  test_size])

# 创建数据加载器
train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

# 创建模型实例和优化器
model = CNNModel()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# 训练模型
for epoch in range(num_epochs):
    train_loss = train_model(model, train_dataloader, criterion, optimizer, device)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {train_loss:.4f}")

# 模型推断
predictions = inference(model, test_dataloader, device)
print("Predictions:", predictions)